In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
import glob

from tqdm import tqdm
import ray

### Step 1. Feature eng

1. 새롭게 만든 파생변수가 논리적이어야 한다.
    - 1.1. Data driven
        - 만든 파생변수가 대출 승인 받았을때와 받지 않았을때의 distribution 차이. plot, SKlearn 
        - EDA 과정 다음 Link 확인 : https://dacon.io/competitions/official/235927/codeshare/6052?page=1&dtype=recent
        - EDA 과정(파생변수 만들기) : https://dacon.io/competitions/official/235930/codeshare/6002?page=1&dtype=recent 
        
    - 1.2. Stat driven
        - 만든 파생변수가 통계적으로 is_applied(Y/N)에서 차이가 있는지 검정.
        - Numeric 한 변수면 -> 만든 파생변수가 정규분포를 따르는지 확인 샤피로 웰콘슨 테스트(정규분포 테스트) -> t-test
        - Numeric 하지 않으면 -> 비모수검정, 맨 흴트이 윌콕슨 테스트 등
        
2. 만든 파생변수 N개를 가지고 간단한 베이스라인 ML 모델로 확인 : https://dacon.io/competitions/official/235930/codeshare/6002?page=1&dtype=recent
    - Step 1. 기본 변수로만 사용해서 성능확인 // 파생변수 N개만 사용해서 성능확인 -> 각각 2개확인
    - Step 2. 기본 변수로만 사용해서 성능확인 + 파생변수 1개씩 성능확인 (총 nCk 조합)
    
    
### Q1. Loan_limit, loan_rate가 중요한가?

- 대출이 승인되었는지 안되었는지를 예측하기 위해서 loan_limit & roan_rate가 중요한지 고민해보기
- "우선은 중요하지 않다" 라고 가정하고 결측치는 제거
- 대체적으로 is_applied가 1로 나타나는데 이는 label noise인가? 만약 label noise라면 확인할방법은?


## Label encoding 

1. income_type : `['EARNEDINCOME' 'EARNEDINCOME2' 'FREELANCER' 'OTHERINCOME' 'PRACTITIONER' 'PRIVATEBUSINESS']`

2. employment_type : `['계약직' '기타' '일용직' '정규직']`

3. houseown_type : `['기타가족소유' '배우자' '자가' '전월세']`

4. purpose : `['기타' '대환대출' '사업자금' '생활비' '자동차구입' '전월세보증금' '주택구입' '투자']`


### 변수 description
- Categorical variables
    - cat_already = `["bank_id", "product_id", "gender"]`
    - cat = `["income_type", "employment_type", "houseown_type", "purpose"]`
 
- Categorical로 만들어볼만한 변수
     - cont = `["loan_limit", "loan_rate", "birth_year", "credit_score", "yearly_income", "company_enter_month", "desired_amount" ,"existing_loan_cnt","existing_loan_amt"]`
 
- 쓸모없을것 같은 변수 User ID, application_id는 반드시 삭제
    - del_var = `["loanapply_insert_time", "application_id", "insert_time", "user_id"]`
 
- 타겟 변수
    - val_y = `["is_applied"]`

In [2]:
full_data = pd.read_csv("./prepro_data/full_data.csv")
submit_test = pd.read_csv("./prepro_data/submit_test.csv")
dataset = pd.concat([full_data, submit_test])

### User Behavior data

In [3]:
log_data = pd.read_csv("./data/log_data.csv")
user_spec = pd.read_csv("./data/user_spec.csv")

In [4]:
dataset["YYYY_MM_DD"] = dataset["loanapply_insert_time"].str[:10]
user_spec["YYYY_MM_DD"] = user_spec["insert_time"].str[:10]

In [5]:
dataset = dataset[["application_id", "YYYY_MM_DD", "user_id", "loanapply_insert_time"]]
user_spec = user_spec[["application_id", "user_id","YYYY_MM_DD", "insert_time"]]



In [6]:
dataset_0= np.sort(dataset["user_id"].unique())[0:30000]
dataset_1= np.sort(dataset["user_id"].unique())[30000:60000]
dataset_2= np.sort(dataset["user_id"].unique())[60000:90000]
dataset_3= np.sort(dataset["user_id"].unique())[90000:120000]
dataset_4= np.sort(dataset["user_id"].unique())[120000:150000]
dataset_5= np.sort(dataset["user_id"].unique())[150000:180000]
dataset_6= np.sort(dataset["user_id"].unique())[180000:210000]
dataset_7= np.sort(dataset["user_id"].unique())[210000:240000]
dataset_8= np.sort(dataset["user_id"].unique())[240000:270000]
dataset_9= np.sort(dataset["user_id"].unique())[270000:300000]
dataset_10= np.sort(dataset["user_id"].unique())[300000:]


In [7]:
dataset_df_0 =dataset[dataset["user_id"].isin(dataset_0)]
dataset_df_1 =dataset[dataset["user_id"].isin(dataset_1)]
dataset_df_2 =dataset[dataset["user_id"].isin(dataset_2)]
dataset_df_3 =dataset[dataset["user_id"].isin(dataset_3)]
dataset_df_4 =dataset[dataset["user_id"].isin(dataset_4)]
dataset_df_5 =dataset[dataset["user_id"].isin(dataset_5)]
dataset_df_6 =dataset[dataset["user_id"].isin(dataset_6)]
dataset_df_7 =dataset[dataset["user_id"].isin(dataset_7)]
dataset_df_8 =dataset[dataset["user_id"].isin(dataset_8)]
dataset_df_9 =dataset[dataset["user_id"].isin(dataset_9)]
dataset_df_10 =dataset[dataset["user_id"].isin(dataset_10)]

user_spec_df_0 = user_spec[user_spec["user_id"].isin(dataset_0)]
user_spec_df_1 = user_spec[user_spec["user_id"].isin(dataset_1)]
user_spec_df_2 = user_spec[user_spec["user_id"].isin(dataset_2)]
user_spec_df_3 = user_spec[user_spec["user_id"].isin(dataset_3)]
user_spec_df_4 = user_spec[user_spec["user_id"].isin(dataset_4)]
user_spec_df_5 = user_spec[user_spec["user_id"].isin(dataset_5)]
user_spec_df_6 = user_spec[user_spec["user_id"].isin(dataset_6)]
user_spec_df_7 = user_spec[user_spec["user_id"].isin(dataset_7)]
user_spec_df_8 = user_spec[user_spec["user_id"].isin(dataset_8)]
user_spec_df_9 = user_spec[user_spec["user_id"].isin(dataset_9)]
user_spec_df_10 =user_spec[user_spec["user_id"].isin(dataset_10)]

In [8]:
dataset_np_0 =  dataset_df_0.to_numpy()
dataset_np_1 =  dataset_df_1.to_numpy()
dataset_np_2 =  dataset_df_2.to_numpy()
dataset_np_3 =  dataset_df_3.to_numpy()
dataset_np_4 =  dataset_df_4.to_numpy()
dataset_np_5 =  dataset_df_5.to_numpy()
dataset_np_6 =  dataset_df_6.to_numpy()
dataset_np_7 =  dataset_df_7.to_numpy()
dataset_np_8 =  dataset_df_8.to_numpy()
dataset_np_9 =  dataset_df_9.to_numpy()
dataset_np_10 = dataset_df_10.to_numpy()

In [9]:
@ray.remote
def behavior_l(data, user_data, dataset_np):
    l = []
    j = []
    k = []
    for i in range(data.shape[0]):
        u = user_data[(user_data["user_id"] == dataset_np[i][2])]
        d = data[(data["user_id"] == dataset_np[i][2])]
        
        l.append( len(set(u[u["YYYY_MM_DD"] <= dataset_np[i][1]]["application_id"])) - len(set(d[d["YYYY_MM_DD"] <= dataset_np[i][1]]["application_id"])) )
                 
    return l
    
    

In [10]:
@ray.remote
def behavior_j(data, user_data, dataset_np):
    j = []
    for i in range(data.shape[0]):
        u = user_data[(user_data["user_id"] == dataset_np[i][2])]
        d = data[(data["user_id"] == dataset_np[i][2])]
        
        j.append( len(set(u[u["YYYY_MM_DD"] <= dataset_np[i][1]]["application_id"]))) # 총 승인할려고한 햇수
                 
    return j
    
    

In [11]:
@ray.remote
def behavior_k(data, user_data, dataset_np):
    k = []
    for i in range(data.shape[0]):
        u = user_data[(user_data["user_id"] == dataset_np[i][2])]
        d = data[(data["user_id"] == dataset_np[i][2])]
    
        k.append( len(set(d[d["YYYY_MM_DD"] <= dataset_np[i][1]]["application_id"]))) # 총 승인된 횟수
                 
    return k
    
    

In [12]:
dataset_put_0 = ray.put(dataset_df_0)
user_spec_put_0 = ray.put(user_spec_df_0)
dataset_np_put_0 = ray.put(dataset_np_0)

dataset_put_1 = ray.put(dataset_df_1)
user_spec_put_1 = ray.put(user_spec_df_1)
dataset_np_put_1 = ray.put(dataset_np_1)

dataset_put_2 = ray.put(dataset_df_2)
user_spec_put_2 = ray.put(user_spec_df_2)
dataset_np_put_2 = ray.put(dataset_np_2)

dataset_put_3 = ray.put(dataset_df_3)
user_spec_put_3 = ray.put(user_spec_df_3)
dataset_np_put_3 = ray.put(dataset_np_3)

dataset_put_4 = ray.put(dataset_df_4)
user_spec_put_4 = ray.put(user_spec_df_4)
dataset_np_put_4 = ray.put(dataset_np_4)

dataset_put_5 = ray.put(dataset_df_5)
user_spec_put_5 = ray.put(user_spec_df_5)
dataset_np_put_5 = ray.put(dataset_np_5)

dataset_put_6 = ray.put(dataset_df_6)
user_spec_put_6 = ray.put(user_spec_df_6)
dataset_np_put_6 = ray.put(dataset_np_6)

dataset_put_7 = ray.put(dataset_df_7)
user_spec_put_7 = ray.put(user_spec_df_7)
dataset_np_put_7 = ray.put(dataset_np_7)

dataset_put_8 = ray.put(dataset_df_8)
user_spec_put_8 = ray.put(user_spec_df_8)
dataset_np_put_8 = ray.put(dataset_np_8)

dataset_put_9 = ray.put(dataset_df_9)
user_spec_put_9 = ray.put(user_spec_df_9)
dataset_np_put_9 = ray.put(dataset_np_9)

dataset_put_10 = ray.put(dataset_df_10)
user_spec_put_10 = ray.put(user_spec_df_10)
dataset_np_put_10 = ray.put(dataset_np_10)


result0 =  behavior_l.remote(dataset_put_0, user_spec_put_0, dataset_np_put_0)
result1 =  behavior_l.remote(dataset_put_1, user_spec_put_1, dataset_np_put_1)
result2 =  behavior_l.remote(dataset_put_2, user_spec_put_2, dataset_np_put_2)
result3 =  behavior_l.remote(dataset_put_3, user_spec_put_3, dataset_np_put_3)
result4 =  behavior_l.remote(dataset_put_4, user_spec_put_4, dataset_np_put_4)
result5 =  behavior_l.remote(dataset_put_5, user_spec_put_5, dataset_np_put_5)
result6 =  behavior_l.remote(dataset_put_6, user_spec_put_6, dataset_np_put_6)
result7 =  behavior_l.remote(dataset_put_7, user_spec_put_7, dataset_np_put_7)
result8 =  behavior_l.remote(dataset_put_8, user_spec_put_8, dataset_np_put_8)
result9 =  behavior_l.remote(dataset_put_9, user_spec_put_9, dataset_np_put_9)
result10 = behavior_l.remote(dataset_put_10, user_spec_put_10, dataset_np_put_10)

z0 = ray.get(result0)
z1 = ray.get(result1)
z2 = ray.get(result2)
z3 = ray.get(result3)
z4 = ray.get(result4)
z5 = ray.get(result5)
z6 = ray.get(result6)
z7 = ray.get(result7)
z8 = ray.get(result8)
z9 = ray.get(result9)
z10 = ray.get(result10)

In [13]:
dataset_put_0 = ray.put(dataset_df_0)
user_spec_put_0 = ray.put(user_spec_df_0)
dataset_np_put_0 = ray.put(dataset_np_0)

dataset_put_1 = ray.put(dataset_df_1)
user_spec_put_1 = ray.put(user_spec_df_1)
dataset_np_put_1 = ray.put(dataset_np_1)

dataset_put_2 = ray.put(dataset_df_2)
user_spec_put_2 = ray.put(user_spec_df_2)
dataset_np_put_2 = ray.put(dataset_np_2)

dataset_put_3 = ray.put(dataset_df_3)
user_spec_put_3 = ray.put(user_spec_df_3)
dataset_np_put_3 = ray.put(dataset_np_3)

dataset_put_4 = ray.put(dataset_df_4)
user_spec_put_4 = ray.put(user_spec_df_4)
dataset_np_put_4 = ray.put(dataset_np_4)

dataset_put_5 = ray.put(dataset_df_5)
user_spec_put_5 = ray.put(user_spec_df_5)
dataset_np_put_5 = ray.put(dataset_np_5)

dataset_put_6 = ray.put(dataset_df_6)
user_spec_put_6 = ray.put(user_spec_df_6)
dataset_np_put_6 = ray.put(dataset_np_6)

dataset_put_7 = ray.put(dataset_df_7)
user_spec_put_7 = ray.put(user_spec_df_7)
dataset_np_put_7 = ray.put(dataset_np_7)

dataset_put_8 = ray.put(dataset_df_8)
user_spec_put_8 = ray.put(user_spec_df_8)
dataset_np_put_8 = ray.put(dataset_np_8)

dataset_put_9 = ray.put(dataset_df_9)
user_spec_put_9 = ray.put(user_spec_df_9)
dataset_np_put_9 = ray.put(dataset_np_9)

dataset_put_10 = ray.put(dataset_df_10)
user_spec_put_10 = ray.put(user_spec_df_10)
dataset_np_put_10 = ray.put(dataset_np_10)


result0 =  behavior_j.remote(dataset_put_0, user_spec_put_0, dataset_np_put_0)
result1 =  behavior_j.remote(dataset_put_1, user_spec_put_1, dataset_np_put_1)
result2 =  behavior_j.remote(dataset_put_2, user_spec_put_2, dataset_np_put_2)
result3 =  behavior_j.remote(dataset_put_3, user_spec_put_3, dataset_np_put_3)
result4 =  behavior_j.remote(dataset_put_4, user_spec_put_4, dataset_np_put_4)
result5 =  behavior_j.remote(dataset_put_5, user_spec_put_5, dataset_np_put_5)
result6 =  behavior_j.remote(dataset_put_6, user_spec_put_6, dataset_np_put_6)
result7 =  behavior_j.remote(dataset_put_7, user_spec_put_7, dataset_np_put_7)
result8 =  behavior_j.remote(dataset_put_8, user_spec_put_8, dataset_np_put_8)
result9 =  behavior_j.remote(dataset_put_9, user_spec_put_9, dataset_np_put_9)
result10 = behavior_j.remote(dataset_put_10, user_spec_put_10, dataset_np_put_10)

zz0 = ray.get(result0)
zz1 = ray.get(result1)
zz2 = ray.get(result2)
zz3 = ray.get(result3)
zz4 = ray.get(result4)
zz5 = ray.get(result5)
zz6 = ray.get(result6)
zz7 = ray.get(result7)
zz8 = ray.get(result8)
zz9 = ray.get(result9)
zz10 = ray.get(result10)

In [14]:
dataset_put_0 = ray.put(dataset_df_0)
user_spec_put_0 = ray.put(user_spec_df_0)
dataset_np_put_0 = ray.put(dataset_np_0)

dataset_put_1 = ray.put(dataset_df_1)
user_spec_put_1 = ray.put(user_spec_df_1)
dataset_np_put_1 = ray.put(dataset_np_1)

dataset_put_2 = ray.put(dataset_df_2)
user_spec_put_2 = ray.put(user_spec_df_2)
dataset_np_put_2 = ray.put(dataset_np_2)

dataset_put_3 = ray.put(dataset_df_3)
user_spec_put_3 = ray.put(user_spec_df_3)
dataset_np_put_3 = ray.put(dataset_np_3)

dataset_put_4 = ray.put(dataset_df_4)
user_spec_put_4 = ray.put(user_spec_df_4)
dataset_np_put_4 = ray.put(dataset_np_4)

dataset_put_5 = ray.put(dataset_df_5)
user_spec_put_5 = ray.put(user_spec_df_5)
dataset_np_put_5 = ray.put(dataset_np_5)

dataset_put_6 = ray.put(dataset_df_6)
user_spec_put_6 = ray.put(user_spec_df_6)
dataset_np_put_6 = ray.put(dataset_np_6)

dataset_put_7 = ray.put(dataset_df_7)
user_spec_put_7 = ray.put(user_spec_df_7)
dataset_np_put_7 = ray.put(dataset_np_7)

dataset_put_8 = ray.put(dataset_df_8)
user_spec_put_8 = ray.put(user_spec_df_8)
dataset_np_put_8 = ray.put(dataset_np_8)

dataset_put_9 = ray.put(dataset_df_9)
user_spec_put_9 = ray.put(user_spec_df_9)
dataset_np_put_9 = ray.put(dataset_np_9)

dataset_put_10 = ray.put(dataset_df_10)
user_spec_put_10 = ray.put(user_spec_df_10)
dataset_np_put_10 = ray.put(dataset_np_10)


result0 =  behavior_k.remote(dataset_put_0, user_spec_put_0, dataset_np_put_0)
result1 =  behavior_k.remote(dataset_put_1, user_spec_put_1, dataset_np_put_1)
result2 =  behavior_k.remote(dataset_put_2, user_spec_put_2, dataset_np_put_2)
result3 =  behavior_k.remote(dataset_put_3, user_spec_put_3, dataset_np_put_3)
result4 =  behavior_k.remote(dataset_put_4, user_spec_put_4, dataset_np_put_4)
result5 =  behavior_k.remote(dataset_put_5, user_spec_put_5, dataset_np_put_5)
result6 =  behavior_k.remote(dataset_put_6, user_spec_put_6, dataset_np_put_6)
result7 =  behavior_k.remote(dataset_put_7, user_spec_put_7, dataset_np_put_7)
result8 =  behavior_k.remote(dataset_put_8, user_spec_put_8, dataset_np_put_8)
result9 =  behavior_k.remote(dataset_put_9, user_spec_put_9, dataset_np_put_9)
result10 = behavior_k.remote(dataset_put_10, user_spec_put_10, dataset_np_put_10)

zzz0 = ray.get(result0)
zzz1 = ray.get(result1)
zzz2 = ray.get(result2)
zzz3 = ray.get(result3)
zzz4 = ray.get(result4)
zzz5 = ray.get(result5)
zzz6 = ray.get(result6)
zzz7 = ray.get(result7)
zzz8 = ray.get(result8)
zzz9 = ray.get(result9)
zzz10 = ray.get(result10)

In [15]:
full_data = pd.read_csv("./prepro_data/full_data.csv")
submit_test = pd.read_csv("./prepro_data/submit_test.csv")
full_dataset = pd.concat([full_data, submit_test])

In [16]:
#zzz0

In [17]:
dataset

,application_id,YYYY_MM_DD,user_id,loanapply_insert_time
0,2157865,2022-05-09,346970,2022-05-09 08:44:59
1,576643,2022-05-09,545882,2022-05-09 10:54:53
2,576643,2022-05-09,545882,2022-05-09 10:54:53
3,2136706,2022-05-09,558819,2022-05-09 10:41:06
4,2136706,2022-05-09,558819,2022-05-09 10:41:07
...,...,...,...,...
3257234,1428218,2022-06-03,478802,2022-06-03 12:01:58
3257235,1428218,2022-06-03,478802,2022-06-03 12:01:49
3257236,1428218,2022-06-03,478802,2022-06-03 12:01:48
3257237,1428218,2022-06-03,478802,2022-06-03 12:01:48


In [18]:
num_not_approve = pd.DataFrame({"num_not_approve" : z0 + z1 + z2 + z3 + z4 + z5 + z6 + z7 + z8 + z9+ z10})
num_total_approve = pd.DataFrame({"num_total_approve" : zz0 + zz1 + zz2 + zz3 + zz4 + zz5 + zz6 + zz7 + zz8 + zz9+ zz10})
num_approve = pd.DataFrame({"num_approve" : zzz0 + zzz1 + zzz2 + zzz3 + zzz4 + zzz5 + zzz6 + zzz7 + zzz8 + zzz9+ zzz10})

In [19]:
num_total_approve.min()

num_total_approve    1
dtype: int64

In [20]:
full_dataset["num_not_approve"] = num_not_approve
full_dataset["num_total_approve"] = num_total_approve
full_dataset["num_approve"] = num_approve

In [21]:
full_dataset.to_csv("./prepro_data/full_data_wtih_test.csv", index = False)

In [22]:
full_data = full_dataset[full_dataset["loanapply_insert_time"] < "2022-06-01"]
test_submit_data = full_dataset[full_dataset["loanapply_insert_time"] >= "2022-06-01"]

In [23]:
full_data.to_csv("./prepro_data/full_data.csv", index = False)
test_submit_data.to_csv("./prepro_data/submit_test.csv", index = False)

In [24]:
full_data

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,company_enter_month,employment_type,houseown_type,desired_amount,purpose,existing_loan_cnt,existing_loan_amt,num_not_approve,num_total_approve,num_approve
0,2157865,2022-05-09 08:44:59,54,235,20000000.0,16.5,1.0,346970,1970.0,1.0,...,99,정규직,자가,10000000.0,생활비,12.0,162000000.0,5,7,2
1,576643,2022-05-09 10:54:53,54,235,11000000.0,16.5,0.0,545882,1977.0,1.0,...,26,정규직,전월세,20000000.0,생활비,7.0,106000000.0,5,7,2
2,576643,2022-05-09 10:54:53,11,118,3000000.0,20.0,0.0,545882,1977.0,1.0,...,26,정규직,전월세,20000000.0,생활비,7.0,106000000.0,5,7,2
3,2136706,2022-05-09 10:41:06,42,216,10000000.0,13.5,0.0,558819,1983.0,1.0,...,99,정규직,전월세,80000000.0,대환대출,2.0,58000000.0,5,7,2
4,2136706,2022-05-09 10:41:07,25,169,22000000.0,15.9,0.0,558819,1983.0,1.0,...,99,정규직,전월세,80000000.0,대환대출,2.0,58000000.0,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10270006,1369315,2022-05-16 06:40:24,27,148,18000000.0,7.0,0.0,601384,1977.0,0.0,...,11,기타,자가,6000000.0,생활비,3.0,27000000.0,0,1,1
10270007,1369315,2022-05-16 06:40:25,32,257,18000000.0,7.2,0.0,601384,1977.0,0.0,...,11,기타,자가,6000000.0,생활비,3.0,27000000.0,0,1,1
10270008,1369315,2022-05-16 06:40:45,8,31,16000000.0,9.9,0.0,601384,1977.0,0.0,...,11,기타,자가,6000000.0,생활비,3.0,27000000.0,0,1,1
10270009,1369315,2022-05-16 06:40:45,49,39,30000000.0,15.1,0.0,601384,1977.0,0.0,...,11,기타,자가,6000000.0,생활비,3.0,27000000.0,0,1,1


In [25]:
test_submit_data.shape

(3257239, 24)

In [ ]:
3257239 